In [2]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd

# Load API Key from .env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

# List of asteroid IDs (NEOs) you are interested in
neolist = ['2020 CW', '2020 JJ', '2020 QG', '2020 VT4', '2021 SP', '2021 UA1', 
           '2022 FD1', '2022 GQ5', '2022 KP6', '2022 SK4', '2022 UR4', '2023 AV', 
           '2023 BU', '2023 RS', '2023 UR10', '2024 GJ2', '2024 HA', '2024 LH1', 
           '2024 UZ6', '2024 UG9', '2024 XA', '2025 BP6']

class NasaAPINEOClient:
    API_ENDPOINT = "https://api.nasa.gov/neo/rest/v1/neo/"

    def __init__(self, api_key):
        self.api_key = api_key
        self.data = []

    def fetch_neo_details(self, asteroid_id):
        """
        Fetch details of a specific NEO by its asteroid ID.
        """
        response = requests.get(f"{self.API_ENDPOINT}{asteroid_id}", params={'api_key': self.api_key})
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching {asteroid_id}: {response.status_code}")
            return None

    def get_selected_neo_details(self):
        """
        Fetch detailed data for only the NEOs in the 'neolist'.
        """
        for asteroid_id in neolist:
            details = self.fetch_neo_details(asteroid_id)
            if details:
                for approach in details["close_approach_data"]:
                    self.data.append({

                        "name": details["name"],
                        "is_potentially_hazardous": details.get("is_potentially_hazardous_asteroid"),
                        "is_sentry_object": details.get("is_sentry_object"),
                        "diameter_min_meters": details["estimated_diameter"]["meters"]["estimated_diameter_min"],
                        "diameter_max_meters": details["estimated_diameter"]["meters"]["estimated_diameter_max"],
                        "close_approach_date": approach["close_approach_date"],
                        "relative_velocity_km_h": approach["relative_velocity"]["kilometers_per_hour"],
                        "miss_distance_km": approach["miss_distance"]["kilometers"],
                        "orbiting_body": approach["orbiting_body"],
                        "first_observation_date": details["orbital_data"].get("first_observation_date"),
                        "last_observation_date": details["orbital_data"].get("last_observation_date"),
                        "observations_used": details["orbital_data"].get("observations_used"),
                        "orbit_uncertainty": details["orbital_data"].get("orbit_uncertainty"),
                        "minimum_orbit_intersection": details["orbital_data"].get("minimum_orbit_intersection"),
                        "inclination": details["orbital_data"].get("inclination"),
                        "orbital_period": details["orbital_data"].get("orbital_period")
                    })

        return pd.DataFrame(self.data)

if __name__ == "__main__":
    client = NasaAPINEOClient(NASA_API_KEY)
    df = client.get_selected_neo_details()

